> # Neural Network Basic - Week3 과제


## Import Library

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from dataset.mnist import load_mnist

## Load Dataset
- MNIST 

In [2]:
(X_train, Y_train), (X_test, Y_test) = \
    load_mnist(normalize=True, one_hot_label=False)

In [3]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'Y_train shape: {Y_test.shape}')

X_train shape: (60000, 784)
X_test shape: (10000, 784)
Y_train shape: (60000,)
Y_train shape: (10000,)


## Activation Function 
- sigmoid & relu : hidden layer activation function 
- softmax : output layer activation function 

In [4]:
class sigmoid:
    # sigmoid 함수를 작성하세요 
    def forward(x):
        return 1 / (1+np.exp(-x))
    
    # sigmoid 함수의 미분을 작성하세요
    def backward(x):
        return x * (1-x)

In [5]:
class relu:
    # relu 함수를 작성하세요
    def forward(x):
        return np.maximum(0,x)
    
    # relu 함수의 미분을 작성하세요
    def backward(x):
        return np.where(x>0,1,0)

In [6]:
class softmax:
    def forward(z):
        y = []
        for zi in z:
            c = np.max(zi)
            exp_zi = np.exp(zi-c)
            sum_exp_zi = np.sum(exp_zi)
            yi = exp_zi / sum_exp_zi
            y.append(yi)

        return np.array(y)
    
    def backward(p, y) :
        dp = p.copy()
        for dpi, yi in zip(dp, y):
            for k in range(dp.shape[1]):
                if k == yi :
                    dpi[k] -= 1
        return dp

## Loss Function

In [7]:
def cross_entropy(p, y):
    loss = []
    for pi, yi in zip(p, y):
        for k in range(p.shape[1]):
            if k == yi:
                loss.append((-1) * (np.log(pi[k] + 1e-8))) 
    return loss

## Layer

In [8]:
class Layer:
    def __init__(self, input_size, output_size, std=1e-4) :
        self.input_size = input_size
        self.output_size = output_size
        self.bias = np.random.randn(output_size)
        self.weight = np.random.randn(input_size, output_size)*std

## Neural Network
- 각 메소드와 변수들의 역할을 주석으로 달아주세요! 

In [9]:
class CustomNet:
    # CustomNet을 선언할 때 생성되는 값들입니다.
    def __init__(self, lr=0.0001, epoch=500, batch_size=200):
        self.lr = lr # 학습률
        self.epoch = epoch # 학습 반복 횟수 
        self.batch_size = batch_size # 배치 사이즈
        self.loss_function = cross_entropy # 모델 성능 평가 - 손실함수
        self.layers = [] # Layer 리스트
        self.activations = [softmax] # 활성화 함수
        self.nodes = [] # Node 
    
    # Layer를 추가할 때 호출합니다
    def addLayer(self, Layer): 
        self.layers.append(Layer) # Layer 입력받아 리스트에 추가
        if not self.nodes: # 빈 노드인 경우  
            self.nodes.append(np.zeros(Layer.input_size)) # input size 정보 추가
        self.nodes.append(np.zeros(Layer.output_size)) # output size 정보 추가
        
    # Activation Function을 추가할 때 호출합니다
    def addActivation(self, Activation):
        tmp = self.activations.pop() # 활성화 함수 가장 최근 값을 변수에 저장   
        self.activations.append(Activation) # 입력한 활성화 함수 추가
        self.activations.append(tmp) # pop 했던 최근 활성화 함수 다시 추가함으로 유지
        
    # 순전파 함수
    def _forward(self, X):
        self.nodes[0] = X.copy() # X값을 복사해 노드의 첫 요소로 저장
        output = X.copy() 
        for i in range(len(self.layers)): 
            Layer = self.layers[i] 
            Activation = self.activations[i] #활성화 함수
            output = np.dot(self.nodes[i], Layer.weight)  # 노드 가중치 행렬곱
            output = output+ Layer.bias # 편향 더해줌
            output = Activation.forward(output) # 활성화 함수 적용
            self.nodes[i+1] = output # output을 해당 노드 다음 노드 값으로 저장
        return output   
    
    # 역전파 함수
    def _backward(self, X, output, y) :
        for i in reversed(range(len(self.layers))): # 리스트 안 요소 뒤집는 기능
            a = self.nodes[i+1] # 다음 노드 값 변수에 저장
            Layer = self.layers[i] 
            Activation = self.activations[i] 
            
            if i+1 == len(self.layers): # 마지막 층인 경우
                error = Activation.backward(output, y)
            else: # 마지막 층이 아닌 경우
                error *= Activation.backward(a)
            Layer.weight -= np.dot(error.T, self.nodes[i]).T*self.lr/X.shape[0] # 가중치 업데이트
            Layer.bias -= error.sum(axis=0)*self.lr/X.shape[0] # 편향 업데이트
            error = np.dot(error, Layer.weight.T) # error 계산
            
    # Accrucy를 반환합니다
    def _accuracy(self, output, y):
        pre_p = np.argmax(output, axis=1) # 열을 따라 가장 큰 값의 인덱스 찾아 저장
        return np.sum(pre_p==y)/y.shape[0] # 예측값과 실제 y값 비교 , 정확도 계산
    
    # 데이터셋에 모델을 fit할때 호출합니다
    def fit(self, X, y, val_X, val_y):
        history = {'val_acc': [],'val_loss': []}
        N = X.shape[0]
        for i in range(self.epoch): # 에포크 수만큼 반복
            for j in range(N//self.batch_size): # 배치 사이즈만큼 실행
                batch_mask = np.random.choice(N, self.batch_size) # 랜덤 생성
                X_batch = X[batch_mask] 
                y_batch = y[batch_mask] 
                output = self._forward(X_batch) # 순전파 계산
                self._backward(X_batch, output, y_batch) # 역전파 계산
            
            #accuracy와 loss를 기록해둡시다
            output = self._forward(val_X) # val_X 순전파 함수 적용 , output 
            history["val_acc"].append(self._accuracy(output, val_y)) 
            history["val_loss"].append(sum(self.loss_function(output, val_y))) 
            
            #중간중간 기록을 찍어볼 때 사용. 적절히 조절해 쓰세요
            if i % 10 == 0:
                print(i, "test accuracy :", history["val_acc"][-1])
                print(i, "test loss     :", history["val_loss"][-1])
        return history

## Customizing
- Network parameter, Layer architecture, Activation function .. 등등 다양한 하이퍼파라미터를 커스터마이징하여 높은 성능에 도달해 봅시다! 

In [10]:
# 하이퍼파라미터를 적절히 조절해 뉴럴넷을 선언하세요
nn = CustomNet(lr=0.005, epoch=200, batch_size=400)

# 원하는 만큼 층과 활성화 함수를 쌓아 주세요. 기본적으로 2Layer를 예시로 적어드립니다
nn.addLayer(Layer(784,100))
nn.addActivation(sigmoid)
nn.addLayer(Layer(100,10))

In [11]:
# 선언한 뉴럴넷의 구조입니다
for layer in nn.layers:
    print(layer.weight.shape, layer.bias.shape)

(784, 100) (100,)
(100, 10) (10,)


In [ ]:
history = nn.fit(X_train, Y_train, X_test, Y_test)

0 test accuracy : 0.1135
0 test loss     : 23081.907139160463
10 test accuracy : 0.1136
10 test loss     : 22926.867574830296


## Accuracy, Loss Visualization
- 자유롭게 Accuracy나 Loss를 시각화하여 확인하고 결과를 확인해 보세요! 

In [ ]:
epoch=200
plt.subplot(1, 2, 1)  
plt.plot(range(epoch),history['val_acc'])
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)  
plt.plot(range(epoch),history['val_loss'])
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()